# Chapter 03-03: 메트릭과 평가 (Metrics and Evaluation)

## 학습 목표
- Accuracy, Precision, Recall, F1 등 주요 분류 메트릭을 이해하고 계산할 수 있다
- Confusion Matrix를 시각화하고 해석할 수 있다
- ROC Curve와 AUC를 이해하고 모델 성능을 평가할 수 있다
- `tf.keras.metrics.Metric`을 상속하여 커스텀 메트릭을 작성할 수 있다

## 목차
1. [수학적 기초](#1.-수학적-기초)
2. [기본 분류 메트릭](#2.-기본-분류-메트릭)
3. [Precision, Recall, F1](#3.-Precision,-Recall,-F1)
4. [Confusion Matrix 시각화](#4.-Confusion-Matrix-시각화)
5. [ROC Curve와 AUC](#5.-ROC-Curve와-AUC)
6. [커스텀 메트릭](#6.-커스텀-메트릭)
7. [정리](#7.-정리)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc

print("TensorFlow 버전:", tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)

## 1. 수학적 기초

### 혼동 행렬 (Confusion Matrix)

|  | 예측 양성 | 예측 음성 |
|--|-----------|----------|
| **실제 양성** | TP (True Positive) | FN (False Negative) |
| **실제 음성** | FP (False Positive) | TN (True Negative) |

### 주요 메트릭 공식

**정확도 (Accuracy):**
$$A = \frac{TP + TN}{TP + TN + FP + FN}$$

**정밀도 (Precision):** 양성으로 예측한 것 중 실제 양성 비율
$$P = \frac{TP}{TP + FP}$$

**재현율 (Recall):** 실제 양성 중 올바르게 예측한 비율
$$R = \frac{TP}{TP + FN}$$

**F1 점수:** Precision과 Recall의 조화 평균
$$F_1 = \frac{2PR}{P + R} = \frac{2 \cdot TP}{2 \cdot TP + FP + FN}$$

> **주의**: Precision과 Recall은 트레이드오프 관계이다. 하나를 올리면 다른 하나가 내려가는 경향이 있다.

## 2. 기본 분류 메트릭

In [ ]:
# ---------------------------------------------------
# Accuracy, BinaryAccuracy, CategoricalAccuracy
# ---------------------------------------------------

print("=== Accuracy 메트릭 종류 ===")

# 이진 분류 예시
y_true_bin = tf.constant([1, 0, 1, 1, 0, 1, 0, 0, 1, 1])
y_pred_bin = tf.constant([0.8, 0.3, 0.7, 0.6, 0.4, 0.9, 0.2, 0.7, 0.8, 0.5])

# BinaryAccuracy: 임계값(threshold)을 기준으로 0/1 변환 후 정확도 계산
binary_acc = tf.keras.metrics.BinaryAccuracy(threshold=0.5)
binary_acc.update_state(y_true_bin, y_pred_bin)
print(f"BinaryAccuracy (threshold=0.5): {binary_acc.result().numpy():.4f}")

# 다중 분류 예시
y_true_cat = tf.constant([0, 1, 2, 1, 0])
y_pred_cat = tf.constant([
    [0.8, 0.1, 0.1],  # 클래스 0 예측 (정답)
    [0.2, 0.7, 0.1],  # 클래스 1 예측 (정답)
    [0.3, 0.2, 0.5],  # 클래스 2 예측 (정답)
    [0.6, 0.3, 0.1],  # 클래스 0 예측 (오답: 실제 1)
    [0.7, 0.2, 0.1],  # 클래스 0 예측 (정답)
])

# SparseCategoricalAccuracy: 정수 레이블 + softmax 확률
sparse_acc = tf.keras.metrics.SparseCategoricalAccuracy()
sparse_acc.update_state(y_true_cat, y_pred_cat)
print(f"SparseCategoricalAccuracy: {sparse_acc.result().numpy():.4f}  (4/5 = 0.8)")

# One-hot 레이블 + softmax 확률
y_true_onehot = tf.one_hot(y_true_cat, depth=3)
cat_acc = tf.keras.metrics.CategoricalAccuracy()
cat_acc.update_state(y_true_onehot, y_pred_cat)
print(f"CategoricalAccuracy (one-hot): {cat_acc.result().numpy():.4f}")

# 상태 초기화 방법
print("\n=== 메트릭 상태 관리 ===")
print("""
# 에포크가 끝난 후 메트릭 초기화
metric.reset_state()  # 또는 metric.reset_states()

# 여러 배치 결과 누적
for batch in dataset:
    metric.update_state(y_true, y_pred)

# 에포크 최종 값 출력
epoch_result = metric.result()
metric.reset_state()  # 다음 에포크를 위해 초기화
""")

## 3. Precision, Recall, F1

In [ ]:
# ---------------------------------------------------
# Precision, Recall, F1Score
# ---------------------------------------------------

# 실제 시나리오: 의료 진단 (양성=1: 질병 있음)
# 높은 Recall 중요: 실제 환자를 놓치면 안 됨

y_true_medical = tf.constant([1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1])
y_pred_medical = tf.constant([1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1])

# tf.keras.metrics 사용
precision_metric = tf.keras.metrics.Precision()
recall_metric = tf.keras.metrics.Recall()

precision_metric.update_state(y_true_medical, y_pred_medical)
recall_metric.update_state(y_true_medical, y_pred_medical)

precision_val = precision_metric.result().numpy()
recall_val = recall_metric.result().numpy()

# F1 = 2 * P * R / (P + R) (수동 계산)
f1_val = 2 * precision_val * recall_val / (precision_val + recall_val + 1e-7)

print("=== 의료 진단 시나리오 ===")
print(f"실제 양성 수: {sum(y_true_medical.numpy())}")
print(f"예측 양성 수: {sum(y_pred_medical.numpy())}")
print()
print(f"Precision (정밀도): {precision_val:.4f}")
print(f"  -> 양성으로 예측한 {sum(y_pred_medical.numpy())}건 중 실제 양성 비율")
print(f"Recall    (재현율): {recall_val:.4f}")
print(f"  -> 실제 양성 {sum(y_true_medical.numpy())}건 중 올바르게 탐지한 비율")
print(f"F1 Score          : {f1_val:.4f}")
print()
print("의료 진단에서는 Recall이 중요: 실제 환자를 놓치는 것(FN)이 위험")
print("스팸 필터에서는 Precision이 중요: 정상 메일을 스팸으로 분류(FP)하면 불편")

# 임계값 변경에 따른 Precision-Recall 트레이드오프
print("\n=== 임계값에 따른 P-R 트레이드오프 ===")
y_pred_probs = tf.constant([0.9, 0.85, 0.3, 0.4, 0.1, 0.7, 0.8, 0.75, 0.2, 0.95,
                             0.6, 0.35, 0.88, 0.15, 0.92])

thresholds = [0.3, 0.5, 0.7, 0.9]
print(f"{'임계값':<8} {'Precision':<12} {'Recall':<12} {'F1':<10}")
print("-" * 45)

for thresh in thresholds:
    p_metric = tf.keras.metrics.Precision(thresholds=thresh)
    r_metric = tf.keras.metrics.Recall(thresholds=thresh)
    p_metric.update_state(y_true_medical, y_pred_probs)
    r_metric.update_state(y_true_medical, y_pred_probs)
    p = p_metric.result().numpy()
    r = r_metric.result().numpy()
    f1 = 2 * p * r / (p + r + 1e-7)
    print(f"{thresh:<8.1f} {p:<12.4f} {r:<12.4f} {f1:<10.4f}")

## 4. Confusion Matrix 시각화

In [ ]:
# ---------------------------------------------------
# MNIST 모델로 Confusion Matrix 시각화
# ---------------------------------------------------

# MNIST 데이터 로드 및 간단한 모델 학습
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape(-1, 784).astype('float32') / 255.0
X_test  = X_test.reshape(-1, 784).astype('float32') / 255.0

# 빠른 실험용 축소 데이터
X_tr = X_train[:8000]
y_tr = y_train[:8000]

tf.random.set_seed(42)
model_cm = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model_cm.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model_cm.fit(X_tr, y_tr, epochs=5, batch_size=64, verbose=0)

# 테스트 셋으로 예측
y_pred_probs = model_cm.predict(X_test[:2000], verbose=0)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = y_test[:2000]

# Confusion Matrix 계산 (sklearn 사용)
cm = confusion_matrix(y_true_classes, y_pred_classes)

# 히트맵으로 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 왼쪽: 절대 빈도
im1 = axes[0].imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
axes[0].set_title('Confusion Matrix (절대 빈도)')
plt.colorbar(im1, ax=axes[0])
tick_marks = np.arange(10)
axes[0].set_xticks(tick_marks)
axes[0].set_yticks(tick_marks)
axes[0].set_xticklabels([str(i) for i in range(10)])
axes[0].set_yticklabels([str(i) for i in range(10)])
for i in range(10):
    for j in range(10):
        axes[0].text(j, i, str(cm[i, j]),
                    ha='center', va='center',
                    color='white' if cm[i, j] > cm.max() / 2 else 'black',
                    fontsize=8)
axes[0].set_ylabel('실제 레이블')
axes[0].set_xlabel('예측 레이블')

# 오른쪽: 정규화 (행 기준 = Recall 기준)
cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)
im2 = axes[1].imshow(cm_normalized, interpolation='nearest', cmap=plt.cm.Greens,
                      vmin=0, vmax=1)
axes[1].set_title('Confusion Matrix (정규화, 행=실제 클래스별 비율)')
plt.colorbar(im2, ax=axes[1])
axes[1].set_xticks(tick_marks)
axes[1].set_yticks(tick_marks)
axes[1].set_xticklabels([str(i) for i in range(10)])
axes[1].set_yticklabels([str(i) for i in range(10)])
for i in range(10):
    for j in range(10):
        axes[1].text(j, i, f'{cm_normalized[i, j]:.2f}',
                    ha='center', va='center',
                    color='white' if cm_normalized[i, j] > 0.5 else 'black',
                    fontsize=8)
axes[1].set_ylabel('실제 레이블')
axes[1].set_xlabel('예측 레이블')

plt.tight_layout()
plt.show()

# 대각선 = 올바른 예측, 오프 대각선 = 오분류
total_acc = np.trace(cm) / cm.sum()
print(f"전체 정확도: {total_acc:.4f}")
print("\n가장 혼동이 많은 클래스 쌍 (오분류 순):")
off_diagonal = [(cm[i, j], i, j) for i in range(10) for j in range(10) if i != j]
off_diagonal.sort(reverse=True)
for count, true_cls, pred_cls in off_diagonal[:5]:
    print(f"  실제 {true_cls} -> 예측 {pred_cls}: {count}건")

## 5. ROC Curve와 AUC

**ROC Curve (Receiver Operating Characteristic)**는 다양한 임계값에서의 TPR과 FPR의 관계를 나타낸다.

$$TPR = Recall = \frac{TP}{TP + FN}, \quad FPR = \frac{FP}{FP + TN}$$

**AUC (Area Under the Curve)**: ROC 곡선 아래 면적
- AUC = 1.0: 완벽한 분류기
- AUC = 0.5: 랜덤 분류기 (대각선)
- AUC = 0.0: 완전히 반대로 예측

In [ ]:
# ---------------------------------------------------
# ROC Curve와 AUC 시각화 (이진 분류)
# ---------------------------------------------------

# 이진 분류를 위한 간단한 데이터 생성
from sklearn.datasets import make_classification

X_bin, y_bin = make_classification(
    n_samples=2000, n_features=20, n_informative=15,
    random_state=42
)
X_bin = X_bin.astype('float32')

# 분할
split = 1500
X_bin_train, X_bin_test = X_bin[:split], X_bin[split:]
y_bin_train, y_bin_test = y_bin[:split], y_bin[split:]

# 이진 분류 모델
tf.random.set_seed(42)
model_roc = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(20,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_roc.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model_roc.fit(X_bin_train, y_bin_train, epochs=10, batch_size=32,
               validation_split=0.2, verbose=0)

# 예측 확률
y_pred_prob = model_roc.predict(X_bin_test, verbose=0).flatten()

# sklearn으로 ROC Curve 계산
fpr, tpr, thresholds = roc_curve(y_bin_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# TF AUC 메트릭
tf_auc = tf.keras.metrics.AUC()
tf_auc.update_state(y_bin_test, y_pred_prob)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve
axes[0].plot(fpr, tpr, color='darkorange', lw=2,
             label=f'ROC Curve (AUC = {roc_auc:.4f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--',
             label='랜덤 분류기 (AUC = 0.5)')
axes[0].fill_between(fpr, tpr, alpha=0.1, color='darkorange')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('FPR (False Positive Rate)')
axes[0].set_ylabel('TPR (True Positive Rate = Recall)')
axes[0].set_title('ROC Curve')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)

# Precision-Recall Curve
from sklearn.metrics import precision_recall_curve, average_precision_score
precision_vals, recall_vals, _ = precision_recall_curve(y_bin_test, y_pred_prob)
avg_precision = average_precision_score(y_bin_test, y_pred_prob)

axes[1].plot(recall_vals, precision_vals, color='purple', lw=2,
             label=f'PR Curve (AP = {avg_precision:.4f})')
axes[1].axhline(y=sum(y_bin_test)/len(y_bin_test), color='navy', linestyle='--',
                label=f'랜덤 분류기 (baseline = {sum(y_bin_test)/len(y_bin_test):.2f})')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve')
axes[1].legend(loc='lower left')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"sklearn AUC: {roc_auc:.4f}")
print(f"TF     AUC: {tf_auc.result().numpy():.4f}")
print("\n불균형 데이터셋에서는 AUC보다 Average Precision(PR-AUC)이 더 신뢰할 수 있는 지표이다")

## 6. 커스텀 메트릭

`tf.keras.metrics.Metric`을 상속하여 사용자 정의 메트릭을 구현할 수 있다.

필수 메서드:
- `__init__`: 상태 변수 초기화 (`add_weight` 사용)
- `update_state`: 배치마다 상태 업데이트
- `result`: 현재까지 누적된 메트릭 값 반환
- `reset_state`: 에포크 후 상태 초기화

In [ ]:
# ---------------------------------------------------
# 커스텀 메트릭: MAPE (Mean Absolute Percentage Error)
# ---------------------------------------------------

class MeanAbsolutePercentageError(tf.keras.metrics.Metric):
    """평균 절대 백분율 오차 (MAPE)
    
    수식: MAPE = (1/N) * sum(|y_true - y_pred| / |y_true|) * 100
    - 퍼센트 단위로 표현되어 직관적
    - y_true가 0에 가까울 때 불안정
    """
    
    def __init__(self, name='mape', **kwargs):
        super().__init__(name=name, **kwargs)
        # 누적 합을 저장할 상태 변수
        self.total = self.add_weight(
            name='total',
            initializer='zeros'
        )
        self.count = self.add_weight(
            name='count',
            initializer='zeros'
        )
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        """배치마다 MAPE 누적"""
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        
        # 절대 백분율 오차 계산
        # y_true = 0인 경우 분모에 작은 값 추가 (epsilon)
        abs_pct_error = tf.abs((y_true - y_pred) / (tf.abs(y_true) + 1e-7)) * 100.0
        
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, tf.float32)
            abs_pct_error = abs_pct_error * sample_weight
        
        # 배치 합산 및 개수 업데이트
        self.total.assign_add(tf.reduce_sum(abs_pct_error))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    
    def result(self):
        """현재까지 누적된 MAPE 반환"""
        return self.total / (self.count + 1e-7)
    
    def reset_state(self):
        """에포크 후 상태 초기화"""
        self.total.assign(0.0)
        self.count.assign(0.0)


# 테스트
mape = MeanAbsolutePercentageError()

# 배치 1
y_true_b1 = tf.constant([100.0, 200.0, 50.0])
y_pred_b1 = tf.constant([110.0, 190.0, 55.0])  # 오차: 10%, 5%, 10%
mape.update_state(y_true_b1, y_pred_b1)
print(f"배치 1 후 MAPE: {mape.result().numpy():.2f}%")

# 배치 2 (누적)
y_true_b2 = tf.constant([80.0, 120.0])
y_pred_b2 = tf.constant([88.0, 108.0])   # 오차: 10%, 10%
mape.update_state(y_true_b2, y_pred_b2)
print(f"배치 2 후 MAPE (누적): {mape.result().numpy():.2f}%")

# 초기화 후 재사용
mape.reset_state()
print(f"초기화 후 MAPE: {mape.result().numpy():.2f}%")

# 모델에 적용
print("\n=== 모델에 커스텀 메트릭 적용 ===")
print("""
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=[
        'mae',
        MeanAbsolutePercentageError()  # 커스텀 메트릭
    ]
)
""")

## 7. 정리

### 메트릭 선택 가이드

| 문제 유형 | 권장 메트릭 | 비고 |
|-----------|------------|------|
| 이진 분류 (균형) | Accuracy, AUC | |
| 이진 분류 (불균형) | Precision, Recall, F1, PR-AUC | Accuracy는 오해 유발 가능 |
| 다중 분류 | Accuracy, Macro/Micro F1 | 클래스별 불균형 주의 |
| 회귀 | MAE, MSE, MAPE, R² | 도메인에 따라 선택 |

### 커스텀 메트릭 작성 체크리스트
1. `tf.keras.metrics.Metric` 상속
2. `__init__`에서 `add_weight`로 상태 변수 초기화
3. `update_state`에서 배치 결과를 상태에 누적
4. `result`에서 최종 메트릭 값 계산하여 반환
5. `reset_state`에서 상태 변수를 0으로 초기화

### 핵심 정리
- **Confusion Matrix**: 클래스별 오분류 패턴을 한눈에 파악
- **ROC-AUC**: 임계값에 독립적인 전반적인 분류 성능 평가
- **PR-AUC**: 불균형 데이터에서 더 신뢰할 수 있는 지표
- 메트릭은 배치마다 `update_state`로 누적하고, 에포크 후 `reset_state`로 초기화한다